Funcion para modificar el dataset para trabajar con efficientnet

In [ ]:
import os
import cv2
import torch
import csv
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

class YOLODataset(Dataset):
    def __init__(self, root_dir, mode="train", transform=None):
        self.image_dir = os.path.join(root_dir, "images", mode)
        self.label_dir = os.path.join(root_dir, "labels", mode)
        self.transform = transform
        self.data = self._load_data()

    def _load_data(self):
        data = []
        for label_file in os.listdir(self.label_dir):
            label_path = os.path.join(self.label_dir, label_file)
            with open(label_path, 'r') as f:
                lines = f.readlines()
                
                # Verificar que haya al menos 2 filas
                if len(lines) < 2:
                    print(f" Advertencia: {label_file} no tiene suficiente información")
                    continue  
                
                parts = lines[1].strip().split()  

                try:
                    class_label = int(parts[0]) - 1  
                    image_path = os.path.join(self.image_dir, label_file.replace('.txt', '.jpg'))
                    
                    if os.path.exists(image_path):  
                        data.append((image_path, class_label))
                except ValueError as e:
                    print(f"Error procesando {label_file}: {e}")

        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

root_dir = "dataset_yolo"

train_dataset = YOLODataset(root_dir, mode="train", transform=transform)
valid_dataset = YOLODataset(root_dir, mode="valid", transform=transform)
test_dataset = YOLODataset(root_dir, mode="test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)




Implementacion de efficientnet para la clasificacion de imagenes

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#  Cargar EfficientNet-B0 preentrenado
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 2)  # 2 clases (binario)
model = model.to(device)

#  Definir función de pérdida y optimizador
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#  Cargar checkpoints si existen
checkpoint_path = "efficientnet_checkpoint.pth"
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    if 'model_state' in checkpoint and 'optimizer_state' in checkpoint:
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        start_epoch = checkpoint['epoch'] + 1
    else:
        print("Checkpoint corrupto. Entrenando desde 0.")
        start_epoch = 0
else:
    start_epoch = 0

#  Archivo CSV para guardar métricas
metrics_file = "efficientnet_metrics.csv"
if not os.path.exists(metrics_file):
    with open(metrics_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Epoca", "Perdida", "Precision (Entrenamiento)", "Precision (Validacion)"])

num_epochs = 10
for epoch in range(start_epoch, num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    model.eval()
    correct_valid = 0
    total_valid = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct_valid += (predicted == labels).sum().item()
            total_valid += labels.size(0)

    valid_accuracy = 100 * correct_valid / total_valid

    print(f"Época {epoch + 1}/{num_epochs}, Pérdida: {train_loss:.4f}, Precisión (Train): {train_accuracy:.2f}%, Precisión (Valid): {valid_accuracy:.2f}%")

    with open(metrics_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([epoch + 1, train_loss, train_accuracy, valid_accuracy])

    #  Guardar checkpoint cada época
    torch.save({
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
    }, checkpoint_path)


